# RBM emulator for coupled channels scattering

We would like to construct an emulator for a toy problem for nucleon-nucleus scattering in which we explicitly include a few excited states of the target in our model space.